In [ ]:
!pip uninstall google.cloud

In [5]:
# 1. Library imports
import uvicorn
from fastapi import FastAPI
#from model import IrisModel, IrisSpecies
from generate_images import *

# 2. Create app and model objects
app = FastAPI()
#model = IrisModel()
model = create_model(Configuration)

# 3. Expose the prediction functionality, make a prediction from the passed
#    JSON data and return the predicted flower species with the confidence
"""
@app.post('/predict')
def predict_species(iris: IrisSpecies):
    data = iris.dict()
    prediction, probability = model.predict_species(
        data['sepal_length'], data['sepal_width'], data['petal_length'], data['petal_width']
    )
    return {
        'prediction': prediction,
        'probability': probability
    }
"""
@app.post('/generate_images')
def generate_images():
    config = Configuration()
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="tensorboard",
        project_dir=os.path.join(config.output_dir, "logs"),
    )

    image_generation_pipeline = initialize_pipeline(accelerator, config)

    images = image_generation_pipeline(
        batch_size=config.eval_batch_size,
        generator=torch.manual_seed(config.seed),
        num_inference_steps=config.number_of_noise_steps,
    ).images  # type: ignore (There are multiple definitions which break the type hints)

    if isinstance(images, list):
        save_images(images)


#suggestion: just use the app.py instead. Notebook + FastAPI is funky.


# 4. Run the API with uvicorn
#    Will run on http://127.0.0.1:8000
if __name__ == '__main__':
    pass
    #uvicorn.run(app, host='127.0.0.1', port=8000)

In [8]:
!pip install google-cloud-storage

   ---------------------------------------- 0.0/125.6 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.6 kB ? eta -:--:--
   ---------------------------------------- 125.6/125.6 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/80.6 kB ? eta -:--:--
   ---------------------------------------- 80.6/80.6 kB 4.4 MB/s eta 0:00:00


# Example IRIS call for prediction

In [3]:
import requests 

new_measurement = {
    'sepal_length': 5.7,
    'sepal_width': 3.1,
    'petal_length': 4.9,
    'petal_width': 2.2
}

#response = requests.post('http://127.0.0.1:8000/predict', json=new_measurement)
#print(response.content)

response = requests.post('http://127.0.0.1:8000/generate_images')
print(response.content)


b'Internal Server Error'


#### Calling the prediction in the cmd

curl -X POST "http://127.0.0.1:8000/predict" -H "accept: application/json" -H "Content-Type: application/json" -d "{\"sepal_length\":5.7,\"sepal_width\":3.1,\"petal_length\":4.9,\"petal_width\":2.2}"


#### returns:
{"prediction":"Virginica","probability":0.93}

curl -X POST "http://127.0.0.1:8000/generate_images"

returns: images in the folder predictions / null after the command. 